In [30]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
import statsmodels.formula.api as smf
import matplotlib as plt
from sklearn.linear_model import LogisticRegression

In [31]:
df= pd.read_excel("framingham.xlsx")

In [32]:
df

,male,age,education,currentSmoker,cigsPerDay,BPMeds,heartRate,BMI,diaBP,totChol,sysBP
0,1,39,4.0,0,0.0,0.0,80.0,26.97,70.0,195.0,106.0
1,0,46,2.0,0,0.0,0.0,95.0,28.73,81.0,250.0,121.0
2,1,48,1.0,1,20.0,0.0,75.0,25.34,80.0,245.0,127.5
3,0,61,3.0,1,30.0,0.0,65.0,28.58,95.0,225.0,150.0
4,0,46,3.0,1,23.0,0.0,85.0,23.10,84.0,285.0,130.0
...,...,...,...,...,...,...,...,...,...,...,...
4233,1,50,1.0,1,1.0,0.0,66.0,25.97,92.0,313.0,179.0
4234,1,51,3.0,1,43.0,0.0,65.0,19.71,80.0,207.0,126.5
4235,0,48,2.0,1,20.0,NaN,84.0,22.00,72.0,248.0,131.0
4236,0,44,1.0,1,15.0,0.0,86.0,19.16,87.0,210.0,126.5


In [33]:
df["male"].value_counts()

0    2419
1    1819
Name: male, dtype: int64

In [34]:
df = df.dropna()
y=df["male"]
X=df[["age","education","currentSmoker","cigsPerDay","BPMeds","heartRate","BMI","diaBP","totChol","sysBP"]]
X=sm.add_constant(X)
logit=sm.Logit(y,X).fit()
logit.summary()
#As a result of logit regression result our equations can be shown as :
# ln(P(Y=1) / P(y=0)) = -1.1530 + 0.0231[age] - -0.5875[currentSmoker] ... -0.0202[sysBP]
# In this equation education variable is not written because it is insignificant for this model.
#Negative coefficient means probability of being male is decresing by these kind of parameters.

Optimization terminated successfully.
         Current function value: 0.592197
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   male   No. Observations:                 3987
Model:                          Logit   Df Residuals:                     3976
Method:                           MLE   Df Model:                           10
Date:                Mon, 23 Aug 2021   Pseudo R-squ.:                  0.1344
Time:                        17:02:45   Log-Likelihood:                -2361.1
converged:                       True   LL-Null:                       -2727.8
Covariance Type:            nonrobust   LLR p-value:                4.007e-151
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -1.1530      0.426     -2.708      0.007      -1.988      -0.318
age               0.0231      0.005      4.803      0.000       0.014       0.033
education         0.0451      0.035      1.281      0.200      -0.024       0.114
currentSmoker    -0.5875      0.122     -4.802      0.000      -0.827      -0.348
cigsPerDay        0.0928      0.006     15.838      0.000       0.081       0.104
BPMeds           -0.6584      0.231     -2.845      0.004      -1.112      -0.205
heartRate        -0.0295      0.003     -9.360      0.000      -0.036      -0.023
BMI               0.0584      0.010      6.099      0.000       0.040       0.077
diaBP             0.0424      0.005      8.120      0.000       0.032       0.053
totChol          -0.0042      0.001     -4.873      0.000      -0.006      -0.003
sysBP            -0.0202      0.003     -6.707      0.000      -0.026      -0.014
=================================================================================
"""

In [35]:
logit.pred_table()

array([[1889.,  371.],
       [ 856.,  871.]])

In [36]:
y_pred = logit.predict(X)

In [37]:
y_pred
#Probablity of Being male for first student is %29.9. Probablity of Being female of second person is %76.6

0       0.299948
1       0.234489
2       0.580751
3       0.916381
4       0.543309
          ...   
4232    0.506337
4233    0.129518
4234    0.940773
4236    0.388701
4237    0.218677
Length: 3987, dtype: float64

In [38]:
y_pred = (logit.predict(X) >= 0.5).astype(int)

In [39]:
y_pred

0       0
1       0
2       1
3       1
4       1
       ..
4232    1
4233    0
4234    1
4236    0
4237    0
Length: 3987, dtype: int32

In [40]:
confusion_matrix(y, y_pred)

array([[1889,  371],
       [ 856,  871]], dtype=int64)

In [41]:
accuracy_score(y, y_pred)

0.6922498118886381

In [42]:
logit = LogisticRegression( solver = "liblinear")
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.84      0.75      2260
           1       0.70      0.50      0.59      1727

    accuracy                           0.69      3987
   macro avg       0.69      0.67      0.67      3987
weighted avg       0.69      0.69      0.68      3987

